# Instalando ElasticSearch

In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.10.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.10.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.10.0-linux-x86_64.tar.gz
!useradd elasticsearch
!chmod -R 777 elasticsearch-7.10.0
!echo "!#/bin/bash" > /content/run.sh
!echo "/content/elasticsearch-7.10.0/bin/elasticsearch &" >> /content/run.sh
!chmod 777 run.sh
# !export JAVA_HOME="/content/elasticsearch-7.10.0/jdk"; sudo -E -u elasticsearch nohup /content/run.sh  > /content/elasticsearch.run.log &
!pip install elasticsearch==7.14.1

^C
useradd: user 'elasticsearch' already exists
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 269, in run
    session = self.get_default_session(options)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 77, in get_default_session
    self._session = self.enter_context(self._build_session(options))
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 93, in _build_session
    index_urls=self._get_index_urls(options),
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/network/session.py", line 275, in __init__
    self.head

In [ ]:
!export JAVA_HOME="/content/elasticsearch-7.10.0/jdk"; sudo -E -u elasticsearch nohup /content/run.sh  > /content/elasticsearch.run.log &

nohup: redirecting stderr to stdout


# Indexando

In [ ]:
index_data = '''{
  "settings": {
    "number_of_shards": 2,
    "number_of_replicas": 1
  },
   "mappings": {
    "dynamic": "true",
    "_source": {
      "enabled": "true"
    },
    "properties": {
      "text": {
        "type": "text"
      },
      "topic": {
        "type": "text"
      }
    }
  }
}'''

f = open("index.json", "w")
f.write(index_data)
f.close()


In [ ]:
# esperar um tempo até o elasticsearch subir
!sleep 40 

In [ ]:
import json
import time
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

client = Elasticsearch()

INDEX_NAME = 'teste'

client.indices.delete(index=INDEX_NAME, ignore=[404])

with open('/content/index.json') as index_file:
  source = index_file.read().strip()
  client.indices.create(index=INDEX_NAME, body=source)

# Etapa 1: Inserindo Dados Textuais para Ranking BM25

https://en.wikipedia.org/wiki/Okapi_BM25

In [ ]:
!wget https://raw.githubusercontent.com/rmarcacini/text-collections/master/complete_texts_csvs/classic4.csv

--2022-07-07 20:19:18--  https://raw.githubusercontent.com/rmarcacini/text-collections/master/complete_texts_csvs/SyskillWebert.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 783193 (765K) [text/plain]
Saving to: ‘SyskillWebert.csv’

SyskillWebert.csv   100%[===================>] 764.84K  --.-KB/s    in 0.008s  

2022-07-07 20:19:18 (95.4 MB/s) - ‘SyskillWebert.csv’ saved [783193/783193]



In [ ]:
import pandas as pd
df_docs = pd.read_csv('classic4.csv')
df_docs

,file_name,text,class
0,53,"LaFond, Lois One World (...",Bands
1,28,"Houston, Penelope Houston, Penelope ...",Bands
2,57,"Bernard, Mary Ellen Bernard, Mary Ellen ...",Bands
3,25,Deth Specula Deth Specula Careenin...,Bands
4,15,Hungry Ghost Hungry Ghost Aboral ...,Bands
...,...,...,...
329,60,Too Busy Too Busy!,Sheep
330,52,Desert Bighorn Sheep Desert Bighorn ...,Sheep
331,39,Wool Manufacturing: Shear Pain Miscella...,Sheep
332,19,Sheep Brain Atlas Saggital View Click on ...,Sheep


In [ ]:
import json
import time
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


#### conectando ao elastic search
client = Elasticsearch()

from tqdm.notebook import tqdm
import pandas as pd
import urllib.request, json 
from tqdm.notebook import tqdm
from pathlib import Path
import time

counter = 0

requests = []
for index,row in df_docs.iterrows():
  request = row.to_dict()
  request["_op_type"] = "index"
  request["_index"] = INDEX_NAME
  requests.append(request)

  if len(requests) >= 10000:
    bulk(client, requests)
    request = []

bulk(client, requests)

BulkIndexError: ignored

### Testando uma query

In [ ]:
import urllib.request, json 

query = urllib.parse.quote_plus('neural networks for linux systems')

L = []
with urllib.request.urlopen("http://localhost:9200/_search/?q="+query+"&size=50") as url:
    data = json.loads(url.read().decode())
    for item in data['hits']['hits']:
      try:
        L.append(item['_source'])
      except:
        1
    #print(data)

df_results = pd.DataFrame(L)
df_results

In [ ]:
import numpy as np
from math import e
import pandas as pd   

def df_entropy(column, base=None):
    vc = pd.Series(column).value_counts(normalize=True, sort=False)
    #print(vc)
    base = len(vc.index)
    entropy = -(vc * np.log(vc)/np.log(base)).sum()
    #print('Entropy: ',entropy)
    return entropy

In [ ]:
df_entropy(df_results['class'])

## Avaliando Múltiplas Queries

Queries candidatas por meio de ngramas

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

In [ ]:
def remove_stopwords(text,stop_words):
  
  # tudo para caixa baixa
  s = str(text).lower() 

  tokens = word_tokenize(s)

  # remove stopwords, dígitos, caracteres especiais e pontuações
  v = [word for word in tokens if not word in stop_words and word.isalnum() and not word.isdigit()]

  return v

def meu_tokenizador(doc, stop_words=nltk.corpus.stopwords.words('english'), stemmer=PorterStemmer()):
  tokens = remove_stopwords(doc,stop_words)
  return tokens

In [ ]:
VSM = TfidfVectorizer(tokenizer=meu_tokenizador,min_df=3,ngram_range=(2,3))
X = VSM.fit_transform(df_docs['text'])

df_ngrams = pd.DataFrame()
df_ngrams['word'] = VSM.get_feature_names()
df_ngrams['tfidf_sum'] = X.toarray().sum(axis=0)
df_ngrams.sort_values(by='tfidf_sum',ascending=False,inplace=True)
df_ngrams.head(50)

In [ ]:
from tqdm.notebook import tqdm

R = []
for index,row in tqdm(df_ngrams.head(1000).iterrows(),total=len(df_ngrams)):


  query = urllib.parse.quote_plus(row['word'])

  L = []
  with urllib.request.urlopen("http://localhost:9200/_search/?q="+query+"&size=50") as url:
      data = json.loads(url.read().decode())
      for item in data['hits']['hits']:
        try:
          L.append(item['_source'])
        except:
          1
      #print(data)
  
  df_results = pd.DataFrame(L)
  entropy = df_entropy(df_results['class'])
  R.append([query,entropy,len(df_results),df_results['class'].to_list()])

In [ ]:
df_results_baseline = pd.DataFrame(R)
df_results_baseline.columns =['query','entropy','size','labels']
df_results_baseline.to_parquet('df_results_baseline_classic4.parquet.gzip',compression='gzip')
df_results_baseline

In [ ]:
df_results_baseline.entropy.describe()

# Etapa 2: Enriquecendo documentos com informação de tópicos

In [ ]:
!pip install bertopic
from bertopic import BERTopic

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer


stopwords_en = nltk.corpus.stopwords.words('english')

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=stopwords_en)

topic_model = BERTopic(embedding_model="all-mpnet-base-v2",
                       nr_topics=300,
                       vectorizer_model=vectorizer_model)


topics = topic_model.fit_transform(df_docs.text.to_list())

In [ ]:
L_topics = []
for t in topics[0]:
  if t!=-1:
    topic_data = topic_model.get_topic(t)
    topic_words = ''
    for v in topic_data:
      topic_words += v[0]+' '
    L_topics.append(topic_words)
  else: L_topics.append('')

df_docs['topic'] = L_topics

df_docs

## Reindexando (com informação de tópico)

In [ ]:
client.indices.delete(index=INDEX_NAME, ignore=[404])

with open('/content/index.json') as index_file:
  source = index_file.read().strip()
  client.indices.create(index=INDEX_NAME, body=source)

counter = 0

requests = []
for index,row in df_docs.iterrows():
  request = row.to_dict()
  request["_op_type"] = "index"
  request["_index"] = INDEX_NAME
  requests.append(request)

  if len(requests) >= 10000:
    bulk(client, requests)
    request = []

bulk(client, requests)

### Testando query

In [ ]:
query = urllib.parse.quote_plus('neural networks for linux systems')

L = []
with urllib.request.urlopen("http://localhost:9200/_search/?q="+query+"&size=50") as url:
    data = json.loads(url.read().decode())
    for item in data['hits']['hits']:
      try:
        L.append(item['_source'])
      except:
        1
    #print(data)

df_results = pd.DataFrame(L)
df_results

In [ ]:
df_entropy(df_results['class'])

In [ ]:
from tqdm.notebook import tqdm

R = []
for index,row in tqdm(df_ngrams.head(1000).iterrows(),total=len(df_ngrams)):


  query = urllib.parse.quote_plus(row['word'])

  L = []
  with urllib.request.urlopen("http://localhost:9200/_search/?q="+query+"&size=50") as url:
      data = json.loads(url.read().decode())
      for item in data['hits']['hits']:
        try:
          L.append(item['_source'])
        except:
          1
      #print(data)
  
  df_results = pd.DataFrame(L)
  entropy = df_entropy(df_results['class'])
  R.append([query,entropy,len(df_results),df_results['class'].to_list()])

In [ ]:
df_results_proposal = pd.DataFrame(R)
df_results_proposal.columns =['query','entropy','size','labels']
df_results_proposal.entropy.describe()

In [ ]:
df_results_proposal.to_parquet('df_results_proposal_classic4.parquet.gzip',compression='gzip')
df_results_proposal

In [ ]:
print('hello')